In [1]:
%pip install -U google-generativeai geopy requests python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.3 MB/s eta 0:00:00? eta -:--:--
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
open-webui 0.6.18 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain-google-genai 2.1.10 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Imports
import google.generativeai as genai
import geopy
from geopy.geocoders import Nominatim
import requests
import json
import os
from dotenv import load_dotenv
from typing import Dict, Any, List
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import Image, display

/Users/lablnet/miniconda3/envs/stat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Import the classes from the rag.py file
from rag import OllamaEmbeddingGenerator, SimpleVectorSearch

In [4]:
# Import the tools from the tools.py file
from tools import get_weather, get_crypto_price, search_in_knowledge_base

📂 Embeddings loaded from hec_outline_embeddings.json
✅ Vector search initialized with 381 documents
🔍 Vector search initialized!


In [5]:
# Import the prompt from the prompt.py file
from prompt import system_prompt

In [11]:
# Load environment variables from .env file
load_dotenv()

# Configure Gemini API
# Method 1: From environment variable (recommended)
api_key = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=api_key)

In [13]:
# Define function schemas for Gemini function calling
function_declarations = [
    genai.protos.FunctionDeclaration(
        name="get_weather",
        description="Get the current weather for a specific location",
        parameters=genai.protos.Schema(
            type=genai.protos.Type.OBJECT,
            properties={
                "location": genai.protos.Schema(
                    type=genai.protos.Type.STRING,
                    description="The location to get weather for (e.g., 'Islamabad', 'New York', 'London')"
                )
            },
            required=["location"]
        )
    ),
    genai.protos.FunctionDeclaration(
        name="get_crypto_price",
        description="Get the current price of a cryptocurrency",
        parameters=genai.protos.Schema(
            type=genai.protos.Type.OBJECT,
            properties={
                "symbol": genai.protos.Schema(
                    type=genai.protos.Type.STRING,
                    description="The cryptocurrency symbol (e.g., 'bitcoin', 'ethereum', 'litecoin', 'dogecoin')"
                )
            },
            required=[]
        )
    ),
    genai.protos.FunctionDeclaration(
        name="search_in_knowledge_base",
        description="Search the knowledge base for information related to course outlines of Pakistan Universities",
        parameters=genai.protos.Schema(
            type=genai.protos.Type.OBJECT,
            properties={
                "query": genai.protos.Schema(
                    type=genai.protos.Type.STRING,
                    description="The search query to find relevant information in the knowledge base"
                )
            },
            required=["query"]
        )
    )
]

In [14]:
# Create the model with function calling enabled
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    tools=function_declarations,
    system_instruction=system_prompt
)

In [41]:
class GeminiAgent:

    def __init__(self, model):
        self.model = model
        self.conversation_history = []
        self.function_map = {
            "get_weather": get_weather,
            "get_crypto_price": get_crypto_price,
            "search_in_knowledge_base": search_in_knowledge_base
        }

    def execute_function(self, function_name: str, args: Dict[str, Any]) -> Any:
        """Execute a function call"""
        if function_name in self.function_map:
            return self.function_map[function_name](**args)
        else:
            return {"error": f"Unknown function: {function_name}"}

    def handle_function_calls(self, parts: List, user_message: str) -> str:
        """Handle multiple function calls recursively"""
        function_calls = []
        function_results = []

        # Extract all function calls from parts
        for part in parts:
            if hasattr(part, 'function_call') and part.function_call:
                function_calls.append(part.function_call)

        if not function_calls:
            return None

        # Execute all function calls
        for function_call in function_calls:
            function_name = function_call.name
            function_args = dict(function_call.args)

            print(f"🔧 Calling tool: {function_name} with {function_args}")

            # Execute the function
            function_result = self.execute_function(function_name, function_args)
            print(f"🔧 TOOL RESPONSE: {function_result}")

            function_results.append({
                "name": function_name,
                "args": function_args,
                "result": function_result
            })

        # Create a new chat session without conversation history to avoid format issues
        new_chat = self.model.start_chat()

        # Build a comprehensive prompt with all function results
        results_text = ""
        for fr in function_results:
            result = fr["result"]
            if isinstance(result, list):
                result = str(result)
            elif not isinstance(result, (str, int, float, bool)):
                result = str(result)

            results_text += f"\n\nFunction {fr['name']} result:\n{result}"

        # Send a comprehensive prompt with all results
        prompt = f"""User asked: {user_message}
        Here are the results from the function calls:{results_text}
        Please provide a comprehensive answer based on these function results."""

        final_response = new_chat.send_message(prompt)

        # Add the conversation to history in a simple format
        self.conversation_history.append({
            "role": "user",
            "parts": [{"text": user_message}]
        })
        self.conversation_history.append({
            "role": "model",
            "parts": [{"text": final_response.text}]
        })

        return final_response.text

    def chat(self, user_message: str) -> str:
        """Main chat method that handles function calling"""
        # Start the chat with existing history
        chat = self.model.start_chat(history=self.conversation_history)

        # Send the message
        response = chat.send_message(user_message)
        print("Raw LLM Response: ", response)

        # Get the parts from the response
        parts = response.candidates[0].content.parts

        # Check if any part contains function calls
        has_function_calls = any(
            hasattr(part, 'function_call') and part.function_call
            for part in parts
        )

        if has_function_calls:
            # Handle function calls (single or multiple)
            return self.handle_function_calls(parts, user_message)
        else:
            # No function call, just return the response
            response_text = response.text

            # Add the response to conversation history using proper format
            self.conversation_history.append({
                "role": "user",
                "parts": [{"text": user_message}]
            })
            self.conversation_history.append({
                "role": "model",
                "parts": [{"text": response_text}]
            })

            return response_text


# Create the agent
agent = GeminiAgent(model)

In [33]:
def stream_conversation(agent, user_message):
    """
    Stream the conversation with the agent
    
    Args:
        agent: The agent to use
        user_message: The message to send to the agent
        
    Returns:
        None
    """
    # Create a path list
    path = ["User", "Agent"]
    
    # Print the title of the conversation
    print("=" * 70)
    print("🎬 GEMINI FUNCTION CALLING AGENT - REAL CONVERSATION")
    print("=" * 70)
    
    # Print the user message
    print(f"\n👤 USER:")
    print(f"   {user_message}")
    
    # Get response from agent
    response = agent.chat(user_message)
    
    # Print the AI response
    print(f"\n🤖 AI: {response}")
    
    print("=" * 70)
    print("✅ CONVERSATION COMPLETE")
    print("=" * 70)

In [26]:
# Test the agent with weather query
stream_conversation(agent, "What's the weather in Islamabad?")

🎬 GEMINI FUNCTION CALLING AGENT - REAL CONVERSATION

👤 USER:
   What's the weather in Islamabad?
Raw LLM Response:  response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_weather",
                  "args": {
                    "location": "Islamabad"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 423,
        "candidates_token_count": 16,
        "total_token_count": 477
      },
      "model_version": "gemini-2.5-flash"
    }),
)
🔧 Calling tool: get_weather with {'location': 'Islamabad'}
🔧 TOOL RESPONSE: {'time': '2025-09-16T10:30', 'interval': 900, 'temperature': 33.1, 'windspe

In [27]:
# Test the agent with crypto query
stream_conversation(agent, "What's the price of Ethereum?")

🎬 GEMINI FUNCTION CALLING AGENT - REAL CONVERSATION

👤 USER:
   What's the price of Ethereum?
Raw LLM Response:  response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_crypto_price",
                  "args": {
                    "symbol": "ethereum"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 590,
        "candidates_token_count": 17,
        "total_token_count": 647
      },
      "model_version": "gemini-2.5-flash"
    }),
)
🔧 Calling tool: get_crypto_price with {'symbol': 'ethereum'}
🔧 TOOL RESPONSE: {'cryptocurrency': 'Ethereum', 'price_usd': 4509.53, 'timestamp': 'current

In [43]:
# Test the agent with knowledge base query
stream_conversation(agent, "What is the pre-requisite for MSCS?")

🎬 GEMINI FUNCTION CALLING AGENT - REAL CONVERSATION

👤 USER:
   What is the pre-requisite for MSCS?
Raw LLM Response:  response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "search_in_knowledge_base",
                  "args": {
                    "query": "MSCS prerequisites admission requirements"
                  }
                }
              },
              {
                "function_call": {
                  "name": "search_in_knowledge_base",
                  "args": {
                    "query": "Master Computer Science eligibility criteria"
                  }
                }
              },
              {
                "function_call": {
                  "name": "search_in_knowledge_base",
                  "args": {
                    "query": "MS